In [1]:
with open(file='/kaggle/input/paul-graham-essays/paul_graham_essays.txt', encoding='utf-8', mode='r') as input_fp:
    text = input_fp.read()
essays = text.split('\n')
print('essay count: {}'.format(len(essays)))

essay count: 55


In [2]:
from arrow import now
from collections import Counter
from nltk import pos_tag
from nltk import word_tokenize
from pandas import DataFrame
from pandas import Series
from plotly.express import bar

time_start = now()
count_df = DataFrame(data=[Series(data=dict(Counter([word for (word, part_of_speech) in pos_tag(word_tokenize(essay)) if part_of_speech in {'NNP', 'NNPS'}]))) for essay in essays]).fillna(value=0)
print('noun count time: {}'.format(now() - time_start))
sum_df = count_df.sum(axis=0).to_frame().reset_index()
bar(data_frame=sum_df.sort_values(ascending=False, by=0).head(n=50), x='index', y=0)

noun count time: 0:00:28.547232


TextBlob unfortunately takes more than an hour to do this task; it produces better results, but entirely in lowercase.

In [3]:
from spacy import load
from spacy.lang.en import stop_words as stop_words


spacy_start = now()
nlp = load('en_core_web_lg')
nlp.max_length = 2655263

data = []
for essay in essays:
    document = nlp(essay)
    count = dict(Counter([chunk.text for chunk in document.noun_chunks]))
    series = Series(data=count)
    data.append(series)
spacy_df = DataFrame(data=data)
print('spacy time: {}'.format(now() - spacy_start))

spacy time: 0:01:52.450417


In [4]:
sum_spacy_df = spacy_df.sum(axis=0).to_frame().reset_index()
bar(data_frame=sum_spacy_df[~sum_spacy_df['index'].str.lower().isin(stop_words.STOP_WORDS)].sort_values(ascending=False, by=0).head(n=50), x='index', y=0)